In [1]:
# Install the kafka-python client
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.3/326.3 kB 3.8 MB/s eta 0:00:00a 0:00:01


iot_sensors

In [ ]:
import time
import csv
import json
from kafka import KafkaProducer

# ---------------------------------------------------------
# Configuration
# ---------------------------------------------------------
# CHANGE THIS PORT TO 29092 (The internal Docker port)
KAFKA_BROKER = 'kafka:29092'  
TOPIC_NAME = 'iot_sensors'
DATA_FILE = '/home/jovyan/data/dataset.csv'

# ---------------------------------------------------------
# Initialize Kafka Producer
# ---------------------------------------------------------
print(f"🔄 Connecting to Kafka at {KAFKA_BROKER}...")
try:
    producer = KafkaProducer(
        bootstrap_servers=[KAFKA_BROKER],
        value_serializer=lambda x: json.dumps(x).encode('utf-8'),
        api_version=(0, 10, 1) # Sometimes helps with version mismatch
    )
    print("✅ Connected to Kafka!")
except Exception as e:
    print(f"❌ Connection Failed: {e}")

# ---------------------------------------------------------
# Start Streaming Data
# ---------------------------------------------------------
print(f"🚀 Starting data stream from {DATA_FILE}...")

try:
    with open(DATA_FILE, 'r') as file:
        reader = csv.DictReader(file) 
        
        count = 0
        for row in reader:
            producer.send(TOPIC_NAME, value=row)
            count += 1
            
            if count % 1000 == 0:
                print(f"📡 Sent {count} records...", end='\r')
            
            time.sleep(0.001) 

except FileNotFoundError:
    print(f"❌ Error: File not found at {DATA_FILE}.")
except KeyboardInterrupt:
    print("\n🛑 Stream stopped by user.")
except Exception as e:
    print(f"❌ Unexpected Error: {e}")
finally:
    if 'producer' in locals():
        producer.flush()
        producer.close()

🔄 Connecting to Kafka at kafka:29092...
✅ Connected to Kafka!
🚀 Starting data stream from /home/jovyan/data/dataset.csv...
